In [1]:
# Lorenz System: Implementation and Analysis

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate
import pandas as pd
import seaborn as sns
from matplotlib.animation import FuncAnimation

In [2]:

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')

In [3]:

# 1. Definition of the Lorenz System
# ---------------------------------
def lorenz_system(t, state, sigma, rho, beta):
    """
    The Lorenz system of differential equations.
    
    Parameters:
    t (float): Time (not used explicitly, but required by scipy.integrate)
    state (array): System state [x, y, z]
    sigma, rho, beta (float): System parameters
    
    Returns:
    list: Derivatives [dx/dt, dy/dt, dz/dt]
    """
    x, y, z = state
    dx_dt = sigma * (y - x)
    dy_dt = x * (rho - z) - y
    dz_dt = x * y - beta * z
    
    return [dx_dt, dy_dt, dz_dt]


In [4]:
# 2. Solving the System with Different Parameters
# ----------------------------------------------
def solve_lorenz(sigma, rho, beta, initial_state, t_span, n_points=10000):
    """
    Solve the Lorenz system with given parameters and initial conditions.
    
    Parameters:
    sigma, rho, beta (float): System parameters
    initial_state (array): Initial state [x0, y0, z0]
    t_span (tuple): Time span for integration (t_start, t_end)
    n_points (int): Number of time points
    
    Returns:
    tuple: (solution object, time array)
    """
    t = np.linspace(t_span[0], t_span[1], n_points)
    
    solution = integrate.solve_ivp(
        lorenz_system, 
        t_span, 
        initial_state, 
        method='RK45',
        t_eval=t,
        args=(sigma, rho, beta)
    )
    
    return solution, t

# Define standard parameters
sigma_standard = 10.0
rho_standard = 50.0
beta_standard = 8.0/3.0

# Initial conditions
initial_state = [1.0, 0.0, 0.0]

# Time span
t_span = (0, 50)


In [5]:
# 3. Visualizing the Lorenz Attractor
# ----------------------------------
# Solve the system with standard parameters
solution_standard, t = solve_lorenz(sigma_standard, rho_standard, beta_standard, initial_state, t_span)

# Extract solution components
x, y, z = solution_standard.y

# Create 3D plot of the trajectory
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the trajectory
ax.plot(x, y, z, color='blue', alpha=0.7, linewidth=0.5)

# Add a colorful scatter plot to show direction of time
scatter = ax.scatter(x, y, z, c=t, cmap='viridis', s=1, alpha=0.5)
fig.colorbar(scatter, ax=ax, label='Time')

# Set labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Lorenz Attractor (σ={}, ρ={}, β={})'.format(sigma_standard, rho_standard, beta_standard))

plt.savefig('lorenz_3d_trajectory.png', dpi=300, bbox_inches='tight')
plt.close()


In [6]:
# 4. Time Series Analysis
# ----------------------
# Plot time series for each variable
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axes[0].plot(t, x, color='blue')
axes[0].set_ylabel('X')
axes[0].set_title('Time Series of Lorenz System Variables')

axes[1].plot(t, y, color='red')
axes[1].set_ylabel('Y')

axes[2].plot(t, z, color='green')
axes[2].set_ylabel('Z')
axes[2].set_xlabel('Time')

plt.tight_layout()
plt.savefig('lorenz_time_series.png', dpi=300, bbox_inches='tight')
plt.close()

In [8]:
# 5. Phase Portraits (Projections)
# ------------------------------
# Create 2D projections of the attractor
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# XY Plane
axes[0].plot(x, y, color='purple', alpha=0.7, linewidth=0.5)
axes[0].scatter(x, y, c=t, cmap='viridis', s=1, alpha=0.5)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title('XY Projection')

# XZ Plane
axes[1].plot(x, z, color='orange', alpha=0.7, linewidth=0.5)
axes[1].scatter(x, z, c=t, cmap='viridis', s=1, alpha=0.5)
axes[1].set_xlabel('X')
axes[1].set_ylabel('Z')
axes[1].set_title('XZ Projection')

# YZ Plane
axes[2].plot(y, z, color='green', alpha=0.7, linewidth=0.5)
axes[2].scatter(y, z, c=t, cmap='viridis', s=1, alpha=0.5)
axes[2].set_xlabel('Y')
axes[2].set_ylabel('Z')
axes[2].set_title('YZ Projection')

plt.tight_layout()
plt.savefig('lorenz_phase_portraits.png', dpi=300, bbox_inches='tight')
plt.close()


In [9]:
# 6. Parameter Variation Analysis
# -----------------------------
# Define different parameter sets to explore
parameter_sets = [
    {"sigma": 10.0, "rho": 28.0, "beta": 8.0/3.0, "name": "Classical Chaos"},
    {"sigma": 10.0, "rho": 14.0, "beta": 8.0/3.0, "name": "Pre-Chaos"},
    {"sigma": 10.0, "rho": 100.0, "beta": 8.0/3.0, "name": "Strong Forcing"},
    {"sigma": 10.0, "rho": 13.926, "beta": 8.0/3.0, "name": "Near Bifurcation"}
]

# Compare 3D trajectories for different parameter sets
fig = plt.figure(figsize=(15, 12))

for i, params in enumerate(parameter_sets):
    # Solve the system
    solution, t_param = solve_lorenz(
        params["sigma"], params["rho"], params["beta"], 
        initial_state, t_span, n_points=5000
    )
    # Extract solution
    x_param, y_param, z_param = solution.y
    
    # Create subplot
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    
    # Plot trajectory
    ax.plot(x_param, y_param, z_param, linewidth=0.5, alpha=0.7)
    
    # Set labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f"{params['name']} (ρ={params['rho']})")

plt.tight_layout()
plt.savefig('lorenz_parameter_comparison.png', dpi=300, bbox_inches='tight')
plt.close()


In [10]:
# 7. Sensitivity to Initial Conditions Analysis
# -------------------------------------------
# Compare trajectories with very slightly different initial conditions
delta = 1e-10  # A tiny perturbation
initial_state_1 = initial_state
initial_state_2 = [initial_state[0] + delta, initial_state[1], initial_state[2]]

# Solve both systems
solution_1, t = solve_lorenz(sigma_standard, rho_standard, beta_standard, initial_state_1, t_span)
solution_2, t = solve_lorenz(sigma_standard, rho_standard, beta_standard, initial_state_2, t_span)

# Calculate Euclidean distance between trajectories over time
distances = np.sqrt(
    (solution_1.y[0] - solution_2.y[0])**2 + 
    (solution_1.y[1] - solution_2.y[1])**2 + 
    (solution_1.y[2] - solution_2.y[2])**2
)

# Plot the logarithm of distance to visualize exponential divergence
plt.figure(figsize=(10, 6))
plt.semilogy(t, distances)
plt.xlabel('Time')
plt.ylabel('Distance between trajectories (log scale)')
plt.title(f'Sensitivity to Initial Conditions (δ={delta})')
plt.grid(True)
plt.savefig('lorenz_sensitivity.png', dpi=300, bbox_inches='tight')
plt.close()

In [11]:
# 8. Fixed Point Analysis
# ---------------------
# Calculate the fixed points for the given parameters
def fixed_points(rho, beta):
    """Calculate the fixed points of the Lorenz system."""
    if rho < 1:
        # Only the origin is a fixed point
        return np.array([[0, 0, 0]])
    else:
        # The origin and two additional fixed points
        c = np.sqrt(beta * (rho - 1))
        return np.array([
            [0, 0, 0],
            [c, c, rho - 1],
            [-c, -c, rho - 1]
        ])

# Get fixed points for standard parameters
fps = fixed_points(rho_standard, beta_standard)

# Plot the attractor with fixed points
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot trajectory
ax.plot(x, y, z, linewidth=0.5, alpha=0.5, color='gray')

# Plot fixed points
ax.scatter(fps[:, 0], fps[:, 1], fps[:, 2], color='red', s=100, label='Fixed Points')

# Labels and title
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Lorenz Attractor with Fixed Points')
ax.legend()

plt.savefig('lorenz_fixed_points.png', dpi=300, bbox_inches='tight')
plt.close()

In [12]:
# 9. Bifurcation Analysis (rho variation)
# ------------------------------------
# Calculate behavior over different values of rho
rho_values = np.linspace(0.5, 30, 30)
max_z_values = []

# For each rho value, solve the system and record maximum z value (as a simple metric)
for rho in rho_values:
    solution, _ = solve_lorenz(sigma_standard, rho, beta_standard, initial_state, t_span, n_points=2000)
    # Discard initial transient
    z_steady = solution.y[2][1000:]
    max_z_values.append(np.max(z_steady))

# Plot bifurcation diagram
plt.figure(figsize=(12, 6))
plt.plot(rho_values, max_z_values, 'ko', markersize=2)
plt.axvline(x=1, color='r', linestyle='--', label='ρ=1 (First bifurcation)')
plt.axvline(x=24.74, color='g', linestyle='--', label='ρ≈24.74 (Onset of chaos)')
plt.xlabel('ρ (Rho)')
plt.ylabel('Maximum Z value')
plt.title('Simple Bifurcation Analysis of Lorenz System')
plt.legend()
plt.grid(True)
plt.savefig('lorenz_bifurcation.png', dpi=300, bbox_inches='tight')
plt.close()



In [13]:
# 10. Local Divergence Rate Estimation (crude approximation of Lyapunov exponent)
# ----------------------------------------------------------------------------
# Calculate local divergence rates
def estimate_local_divergence(x1, y1, z1, x2, y2, z2, dt):
    """Estimate local divergence rate between two nearby trajectories."""
    distances = np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    # Get rid of zeros
    epsilon = 1e-10
    distances = np.maximum(distances, epsilon)
    
    # Calculate rate of divergence
    divergence_rates = np.log(distances[1:] / distances[:-1]) / dt
    
    return divergence_rates

# Use our previously solved trajectories
dt = t[1] - t[0]
divergence_rates = estimate_local_divergence(
    solution_1.y[0], solution_1.y[1], solution_1.y[2],
    solution_2.y[0], solution_2.y[1], solution_2.y[2],
    dt
)

# Plot the distribution of local divergence rates
plt.figure(figsize=(10, 6))
plt.hist(divergence_rates, bins=50, alpha=0.7)
plt.axvline(x=np.mean(divergence_rates), color='r', linestyle='--', 
            label=f'Mean ≈ {np.mean(divergence_rates):.4f} (crude estimate of largest Lyapunov exponent)')
plt.xlabel('Local Divergence Rate')
plt.ylabel('Frequency')
plt.title('Distribution of Local Divergence Rates')
plt.legend()
plt.grid(True)
plt.savefig('lorenz_divergence_rates.png', dpi=300, bbox_inches='tight')
plt.close()



In [14]:
# 11. Statistical Analysis of the Attractor
# --------------------------------------
# Create dataframe with trajectory data
df = pd.DataFrame({
    'X': x,
    'Y': y,
    'Z': z,
    'Time': t
})

# Calculate basic statistics
stats = df[['X', 'Y', 'Z']].describe()
print("Statistical Summary of Lorenz Attractor:")
print(stats)

# Create pair plots to see relationships between variables
sns.pairplot(df[['X', 'Y', 'Z']])
plt.suptitle('Pair Plots of Lorenz System Variables', y=1.02)
plt.savefig('lorenz_pairplot.png', dpi=300, bbox_inches='tight')
plt.close()



Statistical Summary of Lorenz Attractor:
                  X             Y             Z
count  10000.000000  10000.000000  10000.000000
mean       0.699947      0.719869     45.206481
std       10.985646     13.512627     12.509935
min      -24.087049    -35.726475      0.000000
25%       -7.485614     -7.828668     36.081433
50%        1.395363      1.154914     44.117090
75%        9.112975      9.828060     54.882215
max       30.042398     49.687128     90.801486


In [15]:
# 12. Power Spectrum Analysis
# ------------------------
from scipy import signal

# Calculate power spectrum for X variable (after discarding transient)
x_steady = x[1000:]  # Remove initial transient
f, Pxx = signal.welch(x_steady, fs=1.0/(t[1]-t[0]), nperseg=1024)

plt.figure(figsize=(10, 6))
plt.semilogy(f, Pxx)
plt.xlabel('Frequency')
plt.ylabel('Power Spectral Density')
plt.title('Power Spectrum of X Variable')
plt.grid(True)
plt.savefig('lorenz_power_spectrum.png', dpi=300, bbox_inches='tight')
plt.close()


In [17]:

# 13. Create an animation of the trajectory forming (for visualization)
# ----------------------------------------------------------------
def create_lorenz_animation(save_path='lorenz_animation.gif'):
    """Create an animation showing the evolution of the Lorenz trajectory."""
    # Use our solution
    x, y, z = solution_standard.y
    
    # Create figure
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Set limits
    ax.set_xlim(min(x), max(x))
    ax.set_ylim(min(y), max(y))
    ax.set_zlim(min(z), max(z))
    
    # Set labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Lorenz Attractor Formation')
    
    # Create line
    line, = ax.plot([], [], [], linewidth=0.7, color='blue')
    
    # Function to initialize the animation
    def init():
        line.set_data([], [])
        line.set_3d_properties([])
        return line,
    
    # Function to update the animation in each frame
    def update(frame):
        # Plot trajectory up to current frame
        frame = int(frame)  # Ensure frame is an integer
        line.set_data(x[:frame], y[:frame])
        line.set_3d_properties(z[:frame])
        return line,
    
    # Create the animation
    n_frames = len(x)
    step = max(1, n_frames // 200)  # Limit to ~200 frames for performance
    ani = FuncAnimation(fig, update, frames=range(0, n_frames, step),
                        init_func=init, blit=True, interval=20)
    
    # Save animation
    ani.save(save_path, writer='pillow', fps=30, dpi=100)
    plt.close()

# Uncomment to create animation (can take some time)
create_lorenz_animation()



In [ ]:
# 14. Conclusion and Summary
# -----------------------
print("""
Lorenz System Analysis Summary:
-------------------------------
1. We implemented the Lorenz system of differential equations and solved it numerically.
2. We visualized the resulting strange attractor in 3D and in 2D projections.
3. We analyzed time series of the system variables.
4. We investigated the effect of varying parameters, especially ρ (rho).
5. We demonstrated sensitivity to initial conditions (butterfly effect).
6. We identified fixed points and their relationship to the attractor.
7. We performed a simple bifurcation analysis.
8. We estimated local divergence rates (related to Lyapunov exponents).
9. We conducted statistical analysis of the attractor properties.
10. We performed power spectrum analysis to examine frequency components.
11. We created visualizations to aid understanding of this chaotic system.

This system demonstrates key features of chaotic behavior:
- Sensitivity to initial conditions
- Strange attractor with fractal structure
- Deterministic yet unpredictable long-term behavior
- Complex structure arising from simple rules
""")